In [1]:
import pandas as pd
import backtrader as bt
import sys
from loguru import logger

logger.remove()
logger.add(sys.stderr, level="INFO")  # or sys.stdout or other file object
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import datetime

dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H.%M.%S')
data = pd.read_csv("data.csv", parse_dates=['timestamp'], index_col='idx', date_parser=dateparse)
strategy_data = data.resample(rule='15T', on='timestamp').agg(
    {'open': 'first',
     'high': 'max',
     'low': 'min',
     'close': 'last',
     'volume': 'sum'
     })
strategy_data = strategy_data.reset_index()
strategy_data.head(5)

,timestamp,open,high,low,close,volume
0,2022-12-01 00:00:00,17165.44,17194.00,17133.35,17188.29,3008
1,2022-12-01 00:15:00,17188.85,17236.29,17159.61,17202.17,5286
2,2022-12-01 00:30:00,17202.17,17210.53,17151.02,17155.80,2505
3,2022-12-01 00:45:00,17155.80,17179.38,17122.65,17161.55,2565
4,2022-12-01 01:00:00,17161.55,17170.32,17134.07,17145.88,2172


In [10]:
from Bbands import BbandsStrategy
from backtrader.feeds import PandasData

bt_data = PandasData(dataname=strategy_data,
                     datetime='timestamp',
                     high='high',
                     low='low',
                     open='open',
                     close='close',
                     volume='volume')
cerebro = bt.Cerebro()
broker = cerebro.broker
broker.setcash(10000000000.0)
cerebro.adddata(bt_data)
cerebro.optstrategy(BbandsStrategy, period=5, bias=1.1)
cerebro.run()

2023-02-20 14:12:02.673 | INFO     | Bbands:__init__:13 - parameter is 5-1.1


[[<backtrader.cerebro.OptReturn at 0x7f8b54dfdbb0>]]

In [20]:
logger.info('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
logger.info(f'Final Portfolio position: {cerebro.broker.getposition(0)}')

2023-02-20 14:16:06.319 | INFO     | __main__:<module>:1 - Final Portfolio Value: 10000000000.00
2023-02-20 14:16:06.320 | INFO     | __main__:<module>:2 - Final Portfolio position: --- Position Begin
- Size: 0
- Price: 0.0
- Price orig: 0.0
- Closed: 0
- Opened: 0
- Adjbase: None
--- Position End
